# Overview

This is for the second round of testing.  **Updated 26 July 2022**. 

- solved: retrieving multiple pages from Airtable
- Next: using email to try to find individuals.  What do we do if an individual email maps to multiple companies?

In [24]:
import pandas as pd
import numpy as np
import os
import json 
import requests
from airtable import airtable #You need to do a pip install airtable.  Page at https://pypi.org/project/airtable/
from collections import Counter

In [ ]:
#Read the secrets file
def read_secrets(fn) -> dict:
    filename = os.path.join(fn)
    try:
        with open(filename, mode='r') as f:
            return json.loads(f.read())
    except FileNotFoundError:
        return {}

In [ ]:
#note the API base id is specific to the Airtable "base".  So, adjust the secrets and the ID to fit the specific Airtable base.
#the api key is an account-wide value so shouldn't change.
secret = read_secrets('secrets2.json')

In [ ]:
#Start by downloading data from Airtable
#https://towardsdatascience.com/downloading-airtable-data-into-python-89e5c7107a24
#reading participant database - Jayson copy.  Will need to change table id in secrets.json to use the real one.

#Get a table in the Airtable base
#see https://github.com/josephbestjames/airtable.py

at = airtable.Airtable(secret['AIRTABLE_BASE_ID'], secret['AIRTABLE_API_KEY'])

In [ ]:
#Note, only returns 1 page = 100 records per request.  So, we'll need to do pagination.

#Basically I want to write a recursion to make the next call until the current dict size is x<100, then exit.
#But, before that, I'll get two pages and merge?  Or, should I work with them separately?  Could do either, maybe I don't need to merge.

def get_all_records(pages,offset=None,count=0):
    
    count = count + 1
    at_dict = at.get(secret['AIRTABLE_TABLE_ID'],offset=offset) 
    
    if 'offset' in at_dict:
        offset = at_dict['offset']
        pages.append(at_dict) 
        if count % 10 ==0:
            print(count,offset) #this is for debugging.  Remove in production version.
            
    else: #if there is no offset, that means that we hit the last page
        pages.append(at_dict) 
        print(count,offset)
        return(pages)
        exit
        
    return(get_all_records(offset = offset,pages = pages,count=count))

In [ ]:
#For now, need to pass these blank lists.  Might be a better way, like tuples or something.
pages = []
all_pages = get_all_records(pages=pages)

In [18]:
L = len(all_pages)
print(f'{L} pages returned')
p = 0
for i in range(L):
    p = p + len(all_pages[i]['records'])
    
print(f'{p} records total')

'Record ID' in all_pages[125]['records'][0]['fields']

126 pages returned
12546 records total


True

In [19]:
#Check the number of unique email addresses. 
#Only get an email address if there is a Record ID.  Records without a Record ID are duplicates.

L = len(all_pages) #L is the number of "pages".  Need to iterate through each set.
emails = []
for i in range(L):
    for j in range(len(all_pages[i]['records'])):
        f = all_pages[i]['records'][j]['fields']
        if 'Record ID' in f:
            e = all_pages[i]['records'][j]['fields']['Email']
            if len(r) > 5:
                emails.append(e)

In [27]:
print(len(emails))
print(len(Counter(emails).keys()))
#So, we have duplicate emails
unique_emails = Counter(emails).keys()
len(unique_emails)

12535
12449


12449

In [ ]:
# for i in range(rec):
#     f = at_dict['records'][i]['fields']
#     id = at_dict['records'][i]['id']
#     n = at_dict['records'][i]['fields']['Email']
#     if n == 'ayushman.sinhaa@gmail.com':
#         print(i,id)

In [ ]:
#A field to update
#update_data = {"Email": "updated@viaAPI.com"}
#update_data

In [ ]:
#Update the email address for a given record based on the record id (generated by AirTable)
#Here, the id was obtained by matching the record id for name 'Nthabi Choma'
#at.update(secret['AIRTABLE_TABLE_ID'], id, update_data)